In [1]:
! pip install numpy==1.26.0 rasterio==1.3.8 shapely==2.0.1 geopandas==0.14.0 torch==2.0.1 torchvision==0.15.2 git+https://github.com/PyTorchLightning/pytorch-lightning

  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-krgzdj6c
  Running command git clone --filter=blob:none --quiet https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-krgzdj6c
  Resolved https://github.com/PyTorchLightning/pytorch-lightning to commit d1f8b0f7669aa808df5fcc2934455be3c83b4bae
  Running command git submodule update --init --recursive -q
  Encountered 31 file(s) that should have been pointers, but weren't:
        .notebooks/course_UvA-DL/01-introduction-to-pytorch.ipynb
        .notebooks/course_UvA-DL/02-activation-functions.ipynb
        .notebooks/course_UvA-DL/03-initialization-and-optimization.ipynb
        .notebooks/course_UvA-DL/04-inception-resnet-densenet.ipynb
        .notebooks/course_UvA-DL/05-transformers-and-MH-attention.ipynb
        .notebooks/course_UvA-DL/06-graph-neural-networks.ipynb
        .notebooks/course_UvA-DL/07-deep-energy-based-generative-models.ipynb
        .notebooks/course_UvA-DL/08-

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# the folder in Google Drive that contains:
## 'train' folder with training rasters
## 'filter_dem' containing the ArcticDEM preprocessed rasters
## 'train_labels' folder containing rasterized polygons corrosponding to training rasters
## the model checkpoint is also saved in this directory
BASE_FOLDER = '/content/drive/MyDrive/Research/SIGSPATIAL Competition/data'

# the same folder is available here:
# https://drive.google.com/drive/folders/1cGyDLLmZafQGetRfnqp8NPGrozed6nx-?usp=sharing
# you can copy it and update the path to where it is copied in your directory

In [4]:
# ! rm -rf ./labels ./tiles

In [5]:
from __future__ import print_function, division
import torch
from torch import nn
from torch.nn import functional as F
import numpy as np
import os
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torch
import sys
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning.pytorch as pl
from torch.utils.data.sampler import SubsetRandomSampler
from lightning.pytorch.callbacks import ModelCheckpoint
import numpy as np

import rasterio
from glob import glob
import os
import rasterio
import numpy as np
import pandas as pd
import torch
import sys
from glob import glob

In [6]:


output_dir = '.'
print("output directory: "+ output_dir)
os.makedirs(output_dir+'/tiles')
os.makedirs(output_dir+'/labels')

DIM = 256
Step = 1

csv_f = open(output_dir + '/tile_info.csv', 'w')
csv_f.write('region_id\traster_id\tx\ty\tsum\n')
# index = 0
for region_id in range(1,7):
    print(region_id)
    raster_paths = glob(BASE_FOLDER + '/train/region_0%d/*.tif' % region_id)
    dem_path = BASE_FOLDER + '/filter_dem/dem_region_0%d.tif' % region_id

    dem = rasterio.open(dem_path)
    dem_read = dem.read()
    dem_avg = np.mean(dem.read()[dem != -9999.0])



    for raster_path in raster_paths:
        raster_id = raster_path[raster_path.find('_2019')+1:raster_path.find('.tif')]

        label_path = BASE_FOLDER + '/train_labels/region_0%d/Greenland26X_22W_Sentinel2_%s.tif' % (region_id, raster_id)
        labels = rasterio.open(label_path)
        labels_arr = labels.read()

        min_shape_1 = min(labels_arr.shape[1],dem_read.shape[1])
        min_shape_2 = min(labels_arr.shape[2],dem_read.shape[2])
        x_pad = DIM-(min_shape_1%DIM)
        y_pad = DIM-(min_shape_2%DIM)

        labels_arr = np.pad(labels_arr, ((0,0), (0, x_pad), (0, y_pad)), 'reflect')
        dem_arr = np.pad(dem_read, ((0,0), (0, x_pad), (0, y_pad)), 'reflect')
        dem_arr[dem_arr == -9999.0] = dem_avg
        for x in range(0, min_shape_1-DIM+Step, DIM // Step):
            for y in range(0, min_shape_2-DIM+Step, DIM // Step):
                cropped_label = labels_arr[:, x:x+DIM, y:y+DIM]
                cropped_dem = dem_arr[:, x:x+DIM, y:y+DIM]

                if len(cropped_dem) > 0 and cropped_dem.mean() > 800:
                    sum_value = cropped_label.sum()
                    csv_f.write(str(region_id) +  '\t' + raster_id + '\t' + str(x) +'\t' + str(y) + '\t' + str(sum_value) +'\n')
csv_f.close()

output directory: .
1
2
3
4
5
6


In [7]:
seed = 42
def set_seed(seed):
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

set_seed(seed)

dataset_dir = '.'

df = pd.read_csv(dataset_dir + '/tile_info.csv', sep='\t')

nonzero_tiles = df[df['sum'] != 0].sample(frac=1).reset_index(drop=True)
zero_tiles = df[df['sum'] == 0].sample(nonzero_tiles.shape[0]).sample(frac=1).reset_index(drop=True)

val_frac=0.8
split1 = int(np.floor(nonzero_tiles.shape[0] * val_frac))
split2 = int(np.floor(zero_tiles.shape[0] * val_frac))

df_train = pd.concat([nonzero_tiles.iloc[0:split1], zero_tiles[0:split2]]).sample(frac=1).reset_index(drop=True)
df_val = pd.concat([nonzero_tiles.iloc[split1:], zero_tiles[split2:]]).sample(frac=1).reset_index(drop=True)

df_train.to_csv(dataset_dir + '/train_tile_info.csv', index=False, sep='\t')
df_val.to_csv(dataset_dir + '/val_tile_info.csv', index=False, sep='\t')

df_selected = pd.concat([nonzero_tiles, zero_tiles]).sample(frac=1).reset_index(drop=True)
df_selected.sort_values(['region_id', 'raster_id', 'x', 'y'], ascending=True, inplace=True, ignore_index=True)

In [8]:
for i in range(df_selected.shape[0]):
    region_id, raster_id, x, y, sum = df_selected.iloc[i].values

for region_id in range(1,7):
    print(region_id)
    raster_paths = glob(BASE_FOLDER + '/train/region_0%d/*.tif' % region_id)
    dem_path = BASE_FOLDER + '/filter_dem/dem_region_0%d.tif' % region_id

    dem = rasterio.open(dem_path)
    dem_read = dem.read()
    dem_avg = np.mean(dem.read()[dem != -9999.0])



    for raster_path in raster_paths:
        raster_id = raster_path[raster_path.find('_2019')+1:raster_path.find('.tif')]
        raster = rasterio.open(raster_path)

        label_path = BASE_FOLDER + '/train_labels/region_0%d/Greenland26X_22W_Sentinel2_%s.tif' % (region_id, raster_id)
        labels = rasterio.open(label_path)
        labels_arr = labels.read()

        min_shape_1 = min(labels_arr.shape[1],dem_read.shape[1])
        min_shape_2 = min(labels_arr.shape[2],dem_read.shape[2])
        x_pad = DIM-(min_shape_1%DIM)
        y_pad = DIM-(min_shape_2%DIM)

        labels_arr = np.pad(labels_arr, ((0,0), (0, x_pad), (0, y_pad)), 'reflect')
        raster_arr = np.pad(raster.read(), ((0,0), (0, x_pad), (0, y_pad)), 'reflect')
        dem_arr = np.pad(dem_read, ((0,0), (0, x_pad), (0, y_pad)), 'reflect')
        dem_arr[dem_arr == -9999.0] = dem_avg
        dem_arr = ( dem_arr - 28.320312 ) / (2468.8047 - 28.320312)

        selected_tiles = df_selected[(df_selected['region_id'] == region_id) & (df_selected['raster_id'] == raster_id)][['x','y']]
        for x in range(0, min_shape_1-DIM+Step, DIM // Step):
            for y in range(0, min_shape_2-DIM+Step, DIM // Step):
                if ((selected_tiles['x'] == x) & (selected_tiles['y'] == y)).any():
                    cropped_raster = raster_arr[:, x:x+DIM, y:y+DIM]
                    cropped_label = labels_arr[:, x:x+DIM, y:y+DIM]
                    cropped_dem = dem_arr[:, x:x+DIM, y:y+DIM]
                    a1 = cropped_raster.astype('float32')/255.0
                    a2 = cropped_dem.astype('float32')
                    a3 = np.zeros((4,DIM,DIM))
                    a3[0:3] = a1
                    a3[3] = a2
                    tensor1 = torch.from_numpy(a3)
                    tensor2 = torch.from_numpy(cropped_label.astype('int32'))


                    sum_value = cropped_label.sum()
                    torch.save(tensor1, output_dir + '/tiles/%d_%s_%d_%d.pt' % (region_id, raster_id, x, y))
                    torch.save(tensor2, output_dir + '/labels/%d_%s_%d_%d.pt' % (region_id, raster_id, x, y))

1
2
3
4
5
6


In [9]:
from pathlib import Path
from functools import partial
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T, utils
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence

from PIL import Image

import io
import urllib
import pandas as pd
class TileDataset(Dataset):
    def __init__(
        self,
        folder,
        is_train=True
    ):
        super().__init__()
        self.folder = folder
        if is_train:
          self.tile_info = pd.read_csv(folder + '/train_tile_info.csv', sep='\t')
        else:
          self.tile_info = pd.read_csv(folder + '/val_tile_info.csv', sep='\t')

    def __len__(self):
        return self.tile_info.shape[0]

    def __getitem__(self, index):
        region_id, raster_id, x, y, _ = self.tile_info.iloc[index].values
        filename = '%d_%s_%d_%d.pt' % (region_id, raster_id, x, y)
        tile = torch.load(self.folder + '/tiles/' + filename)
        label = torch.load(self.folder + '/labels/' + filename)
        return tile.to(torch.float32), label.to(torch.float32)

torch.set_default_dtype(torch.float32)

class conv_block(nn.Module):
    """
    Convolution Block
    """
    def __init__(self, in_ch, out_ch, dropout=0):
        super(conv_block, self).__init__()

        self.conv = nn.Sequential(
            nn.Dropout(dropout),
            nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True))

    def forward(self, x):

        x = self.conv(x)
        return x


class up_conv(nn.Module):
    """
    Up Convolution Block
    """
    def __init__(self, in_ch, out_ch, dropout=0):
        super(up_conv, self).__init__()
        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2),
            nn.Conv2d(in_ch, out_ch, kernel_size=3, stride=1, padding=1, bias=True),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.up(x)
        return x



class U_Net(nn.Module):
    """
    UNet - Basic Implementation
    Paper : https://arxiv.org/abs/1505.04597
    """
    def __init__(self, in_ch=3, out_ch=1):
        super(U_Net, self).__init__()

        n1 = 64
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]

        self.Maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        #self.Maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        #self.Maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.Conv1 = conv_block(in_ch, filters[0], 0)
        self.Conv2 = conv_block(filters[0], filters[1], 0.1)
        self.Conv3 = conv_block(filters[1], filters[2], 0.1)
        #self.Conv4 = conv_block(filters[2], filters[3], 0.1)
        #self.Conv5 = conv_block(filters[3], filters[4], 0.1)

        #self.Up5 = up_conv(filters[4], filters[3])
        #self.Up_conv5 = conv_block(filters[4], filters[3], 0.1)

        #self.Up4 = up_conv(filters[3], filters[2])
        #self.Up_conv4 = conv_block(filters[3], filters[2], 0.1)

        self.Up3 = up_conv(filters[2], filters[1])
        self.Up_conv3 = conv_block(filters[2], filters[1], 0.1)

        self.Up2 = up_conv(filters[1], filters[0])
        self.Up_conv2 = conv_block(filters[1], filters[0], 0.1)

        self.Conv = nn.Conv2d(filters[0], out_ch, kernel_size=1, stride=1, padding=0)

        self.active = torch.nn.Sigmoid()

    def forward(self, x):

        e1 = self.Conv1(x)

        e2 = self.Maxpool1(e1)
        e2 = self.Conv2(e2)

        e3 = self.Maxpool2(e2)
        e3 = self.Conv3(e3)

        #e4 = self.Maxpool3(e3)
        #e4 = self.Conv4(e4)

        #e5 = self.Maxpool4(e4)
        #e5 = self.Conv5(e5)

        #d5 = self.Up5(e5)
        #d5 = torch.cat((e4, d5), dim=1)

        #d5 = self.Up_conv5(d5)

        #d4 = self.Up4(d5)
        #d4 = torch.cat((e3, d4), dim=1)
        #d4 = self.Up_conv4(d4)

        d3 = self.Up3(e3)#d4
        d3 = torch.cat((e2, d3), dim=1)
        d3 = self.Up_conv3(d3)

        d2 = self.Up2(d3)
        d2 = torch.cat((e1, d2), dim=1)
        d2 = self.Up_conv2(d2)

        d1 = self.Conv(d2)

        out = self.active(d1)

        return out


class LitU_net(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.unet = U_Net(4, 1)

    def forward(self, x):
        return self.unet(x)

    def training_step(self, batch):
        x, y = batch
        y_pred = self.forward(x)
        loss = self.dice_loss(y_pred, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch):
        x, y = batch
        y_pred = self.forward(x)
        loss = self.dice_loss(y_pred, y)
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log("dice_coeff", self.dice_coeff(y_pred, y), on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return y_pred

    #def predict_step(self, batch):
       # x, y = batch
        #y_pred = self.forward(x)
        #return y_pred

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def dice_loss(self, logits, targets):
        intersection = 2*(logits * targets).sum() + 1
        union = (logits + targets).sum() + 1
        dice_loss = 1. - intersection / union

        loss = nn.BCELoss()
        bce_loss = loss(logits, targets)
        return 0.5*dice_loss + 0.5*bce_loss

    def dice_coeff(self,logits, targets):
        intersection = 2*(logits * targets).sum()
        union = (logits + targets).sum()
        if union == 0:
            return 1
        dice_coeff = intersection / union
        return dice_coeff.item()


batch_size = 32
num_workers=12

train_loader = torch.utils.data.DataLoader(TileDataset(dataset_dir, is_train=True), batch_size=batch_size,
                                           num_workers=num_workers,shuffle= True)

val_loader = torch.utils.data.DataLoader(TileDataset(dataset_dir, is_train=False), batch_size=batch_size,
                                           num_workers=num_workers)
checkpoint_callback = ModelCheckpoint(monitor="dice_coeff", mode="max",
    dirpath=BASE_FOLDER,
    filename="unet-{epoch:02d}-{dice_coeff:.2f}")


trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=1000, callbacks=[checkpoint_callback], default_root_dir=dataset_dir)
model = LitU_net()

trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:lightning.pytorch.utilities.rank_zero:You are using a CUDA device ('NVIDIA A100-SXM4-40GB') that has Tensor Cores. To properly 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
